In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import h5py
import anndata
from anndata import AnnData
from pathlib import Path
import scipy.io as sio
import scanpy as sc
from tqdm import tqdm

In [4]:
hest_data_root = '/raid/HEST-1K'
hest_csv = pd.read_csv(os.path.join(hest_data_root, 'HEST_v1_0_0.csv'), index_col = 0)
visium_only = hest_csv[hest_csv['st_technology'] == 'Visium']
visium_only_not_brain = visium_only[visium_only['organ']!='Brain']
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['organ']!='Embryo']
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['organ']!='Lung/Brain']
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['organ']!='Kidney/Brain']
# visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['organ']!='nan']
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['species'] == 'Homo sapiens']
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['nb_genes'] > 10000]
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['id'] !='INT17']
visium_ids = list(visium_only_not_brain['id'])
len(visium_ids)

293

In [5]:
visium_only_not_brain

,id,image_filename,organ,disease_state,oncotree_code,species,patient,st_technology,data_publication_date,license,...,nb_genes,treatment_comment,pixel_size_um_embedded,pixel_size_um_estimated,magnification,fullres_px_width,fullres_px_height,tissue,disease_comment,subseries
dataset_title,,,,,,,,,,,,,,,,,,,,,
Visium CytAssist Gene Expression Libraries of Post-Xenium Human Colon Cancer (FFPE),TENX92,TENX92.tif,Bowel,Cancer,COAD,Homo sapiens,NaN,Visium,2023-10-05,Creative Commons Attribution,...,18085,NaN,0.264583,0.273597,40x,28731,37434,Colon,NaN,"Post-Xenium, Replicate 1"
Visium CytAssist Gene Expression Libraries of Post-Xenium Human Colon Cancer (FFPE),TENX91,TENX91.tif,Bowel,Cancer,COAD,Homo sapiens,NaN,Visium,2023-10-05,Creative Commons Attribution,...,18085,NaN,0.264583,0.273681,40x,28004,38719,Colon,NaN,"Post-Xenium, Replicate 2"
Visium CytAssist Gene Expression Libraries of Post-Xenium Human Colon Cancer (FFPE),TENX90,TENX90.tif,Bowel,Cancer,COAD,Homo sapiens,NaN,Visium,2023-10-05,Creative Commons Attribution,...,18085,NaN,0.264583,0.273561,40x,28731,38161,Colon,NaN,"Control, Replicate 1"
Visium CytAssist Gene Expression Libraries of Post-Xenium Human Colon Cancer (FFPE),TENX89,TENX89.tif,Bowel,Cancer,COAD,Homo sapiens,NaN,Visium,2023-10-05,Creative Commons Attribution,...,18085,NaN,0.264583,0.273740,40x,28004,39077,Colon,NaN,"Control, Replicate 2"
"Human Lung Cancer, 11 mm Capture Area (FFPE)",TENX72,TENX72.tif,Lung,Cancer,LNET,Homo sapiens,NaN,Visium,2023-02-14,Creative Commons Attribution,...,18085,NaN,0.264583,0.273711,40x,38510,32875,Lung,Neuroendocrine Carcinoma either ALUCA of LUCA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A spatially resolved atlas of the human lung characterizes a gland-associated immune niche,MISC17,MISC17.tif,Lung,Treated,NaN,Homo sapiens,,Visium,21 December 2022,Creative Commons Attribution 4.0 International...,...,33538,NaN,NaN,0.441499,40x,38240,21600,Bronchi,NaN,WSA_LngSP10193348
A spatially resolved atlas of the human lung characterizes a gland-associated immune niche,MISC16,MISC16.tif,Lung,Treated,NaN,Homo sapiens,,Visium,21 December 2022,Creative Commons Attribution 4.0 International...,...,17922,NaN,NaN,0.455151,40x,33264,18688,Bronchi,NaN,WSA_LngSP10972072
A spatially resolved atlas of the human lung characterizes a gland-associated immune niche,MISC15,MISC15.tif,Lung,Treated,NaN,Homo sapiens,,Visium,21 December 2022,Creative Commons Attribution 4.0 International...,...,17922,NaN,NaN,0.455063,40x,33264,18688,Bronchi,NaN,WSA_LngSP10972073


In [119]:
visium_ids_take

[0,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 

In [9]:
# Prepare StratifiedKFold
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
visium_only_not_brain = visium_only_not_brain[visium_only_not_brain['id'].isin(visium_ids_take)]
# Perform the split
X = visium_only_not_brain.drop(columns=['organ'])
y = visium_only_not_brain['organ']

folds = []

for train_index, val_index in skf.split(X, y):
    train_set = visium_only_not_brain.iloc[train_index]
    val_set = visium_only_not_brain.iloc[val_index]
    folds.append((list(train_set['id']),list( val_set['id'])))
np.save('/raid/eredekop/071024_ST/data/folds.npy', folds)

/raid/eredekop/anaconda3/envs/py39/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,
/raid/eredekop/anaconda3/envs/py39/lib/python3.7/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [13]:
visium_only_not_brain.iloc[14]['organ']

'Prostate'

In [4]:
# def hvg_selection_and_pooling(exp_paths, n_top_genes = 1000):
#         #input n expression matrices paths, output n expression matrices with only the union of the HVGs
#         hvg_bools = []
#         for d in tqdm(exp_paths):
# #             adata = sio.mmread(d)
# #             adata = adata.toarray()
#             adata = sc.read_h5ad(d)
#             adata = adata.X.T
# #             print(adata.shape)
#             adata = sc.AnnData(X=adata.T, dtype=adata.dtype)

#             # Preprocess the data
#             sc.pp.normalize_total(adata)
#             sc.pp.log1p(adata)
#             sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes)
            
#             #save hvgs
#             hvg = adata.var['highly_variable']
# #             print(hvg)
#             hvg_bools.append(hvg)
        
#         #find union of hvgs
#         hvg_union = hvg_bools[0]
#         for i in range(1, len(hvg_bools)):
# #             print(sum(hvg_union), sum(hvg_bools[i]))
#             hvg_union = hvg_union | hvg_bools[i]

#         print("Number of HVGs: ", hvg_union.sum())
#         #filter expression matrices
#         filtered_exp_mtxs = []
#         for d in tqdm(exp_paths):
#             adata = sc.read_h5ad(d)
#             adata = adata.X.T
# #             print('1', d, adata.shape)
#             adata = adata[hvg_union]
#             print('2', adata.shape)
#             filtered_exp_mtxs.append(adata)

#         return filtered_exp_mtxs, hvg_union



# exp_paths = glob('/raid/HEST-1K/st/*.h5ad')
# filtered_mtx, hvg_union = hvg_selection_and_pooling(exp_paths[:10])
# # np.save("/raid/HEST-1K/071524_hvg_matrix/hvg_matrix_{0}.npy".format(exp_paths.split('/')[::-1][0].split('.')[0]), filtered_mtx)
# # np.save("/raid/HEST-1K/071524_hvg_union/hvg_union_{0}.npy".format(exp_paths.split('/')[::-1][0].split('.')[0]), hvg_union)

### first find interestion of genes between all samples

In [7]:

genes = []
for id_ in tqdm(visium_ids):
#     print(id_)
    d = '/raid/HEST-1K/st/{0}.h5ad'.format(id_)
    adata = sc.read_h5ad(d)
    genes.append(list(adata.var_names))
    del adata

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 293/293 [00:33<00:00,  8.70it/s]


In [8]:
visium_ids_take = [visium_ids[0]]
intersection_set = set(genes[0])
# print(intersection_set)
# Perform intersection with subsequent lists
for i, gene_list in enumerate(genes[1:]):
    if len(intersection_set&set(gene_list)) > 10000:
        
        intersection_set &= set(gene_list)
        visium_ids_take.append(visium_ids[i+1])
    else:
        print(i, visium_only_not_brain.iloc[i+1]['organ'], len(intersection_set))

# Convert the set back to a list
intersection_list = list(intersection_set)
len(intersection_list)

18 Breast 17368
104 Lung 17197
176 Uterus 17107
177 Uterus 17107
178 Uterus 17107
179 Uterus 17107
180 Uterus 17107
181 Uterus 17107
182 Uterus 17107
183 Uterus 17107
184 Uterus 17107
185 Uterus 17107
186 Uterus 17107
187 Uterus 17107
188 Uterus 17107
189 Uterus 17107
190 Uterus 17107


16958

In [63]:
n_top_genes = 50
hvg_bools = []
for id_ in tqdm(visium_ids_take):
    try:
#             adata = sio.mmread(d)
#             adata = adata.toarray()
        d = '/raid/HEST-1K/st/{0}.h5ad'.format(id_)
        adata = sc.read_h5ad(d)
        mask = adata.var_names.isin(intersection_list)
        adata = adata[:, mask]

        adata = adata.X.T
    #             print(adata.shape)
        adata = sc.AnnData(X=adata.T, dtype=adata.dtype)

        # Preprocess the data
        sc.pp.normalize_total(adata)
        sc.pp.log1p(adata)
        sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes)

        #save hvgs
        hvg = adata.var['highly_variable']
    #             print(hvg)
        hvg_bools.append(hvg)
    except:
        pass
len(hvg_bools)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 277/277 [06:06<00:00,  1.32s/it]


275

In [64]:
hvg_union = hvg_bools[0]
for i in range(1, len(hvg_bools)):
#     print(sum(hvg_union), sum(hvg_bools[i]))
    hvg_union = hvg_union | hvg_bools[i]

print("Number of HVGs: ", hvg_union.sum())

Number of HVGs:  7968


In [126]:
visium_ids_take

['TENX92',
 'TENX91',
 'TENX90',
 'TENX89',
 'TENX72',
 'TENX71',
 'TENX70',
 'TENX68',
 'TENX65',
 'TENX62',
 'TENX53',
 'TENX51',
 'TENX50',
 'TENX49',
 'TENX46',
 'TENX41',
 'TENX40',
 'TENX39',
 'TENX29',
 'TENX16',
 'TENX15',
 'TENX14',
 'TENX13',
 'ZEN49',
 'ZEN48',
 'ZEN47',
 'ZEN46',
 'ZEN45',
 'ZEN44',
 'ZEN43',
 'ZEN42',
 'ZEN41',
 'ZEN40',
 'ZEN39',
 'ZEN38',
 'ZEN37',
 'ZEN36',
 'INT35',
 'INT28',
 'INT27',
 'INT26',
 'INT25',
 'INT24',
 'INT23',
 'INT22',
 'INT21',
 'INT20',
 'INT19',
 'INT18',
 'INT17',
 'INT16',
 'INT15',
 'INT14',
 'INT13',
 'INT12',
 'INT11',
 'INT10',
 'INT9',
 'INT8',
 'INT7',
 'INT6',
 'INT5',
 'INT4',
 'INT3',
 'INT2',
 'INT1',
 'MEND162',
 'MEND161',
 'MEND160',
 'MEND159',
 'MEND158',
 'MEND157',
 'MEND156',
 'MEND154',
 'MEND153',
 'MEND152',
 'MEND151',
 'MEND150',
 'MEND149',
 'MEND148',
 'MEND147',
 'MEND146',
 'MEND145',
 'MEND144',
 'MEND143',
 'MEND142',
 'MEND141',
 'MEND140',
 'MEND139',
 'MEND96',
 'MEND95',
 'MEND93',
 'MEND92',
 'MEND

In [128]:
filtered_exp_mtxs = []
for id_ in tqdm(visium_ids_take):
    try:
        d = '/raid/HEST-1K/st/{0}.h5ad'.format(id_)
        adata = sc.read_h5ad(d)
    #     print(list(adata.var_names))
        adata = adata.X.T 
    #     print('1', adata.shape)
        adata = adata[hvg_union].toarray()
        print(id_, adata.shape)
        np.save("/raid/HEST-1K/071524_hvg_matrix/hvg_matrix_{0}.npy".format(id_), adata)
    except:
        pass
#     print('2', adata.shape)
#     filtered_exp_mtxs.append(adata)

  0%|                                                                                                                                            | 0/277 [00:00<?, ?it/s]

TENX92 (7968, 6518)


  0%|▍                                                                                                                                   | 1/277 [00:00<03:57,  1.16it/s]

TENX91 (7968, 6352)


  1%|▉                                                                                                                                   | 2/277 [00:02<05:09,  1.12s/it]

...

MISC16 (7968, 3154)
MISC15 (7968, 2501)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 276/277 [01:38<00:00,  3.03it/s]

MISC14 (7968, 1670)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 277/277 [01:38<00:00,  2.82it/s]

MISC13 (7968, 1768)


In [108]:
np.load("/raid/HEST-1K/071524_hvg_matrix/hvg_matrix_{0}.npy".format('TENX92'), allow_pickle=True).toarray()

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [83]:
len(glob('/raid/mpleasure/data_deepstorage/st_projects/visium/uni_embeds_224_20x_normalized/INT1/*'))

1084

In [84]:
df = pd.read_csv('/raid/mpleasure/data_deepstorage/st_projects/visium/all_coords_visium_data.csv')

In [85]:
df[df['id'] == 'INT1'].shape

(1084, 7)

In [86]:
df

,Unnamed: 0,id,bead,patch_x,patch_y,bead_center_x,bead_center_y
0,0,TENX92,0,19979,6911,20091.000000,7023.000000
1,1,TENX92,1,16553,21151,16665.000000,21263.000000
2,2,TENX92,2,23336,27137,23448.000000,27249.000000
3,3,TENX92,3,17099,20201,17211.000000,20313.000000
4,4,TENX92,4,7016,11065,7128.000000,11177.000000
...,...,...,...,...,...,...,...
1337611,3455,MISC1,3455,7810,6263,7922.034775,6375.541625
1337612,3456,MISC1,3456,5374,9477,5486.172801,9589.796448
1337613,3457,MISC1,3457,4545,9711,4657.957578,9823.488712
1337614,3458,MISC1,3458,4353,7915,4465.720971,8027.292313


In [116]:
len(np.unique(visium_ids_take))

276

In [130]:
folds = np.load('/raid/eredekop/071024_ST/data/folds.npy', allow_pickle=True)
df = pd.read_csv('/raid/mpleasure/data_deepstorage/st_projects/visium/all_coords_visium_data.csv')
train_df = df[df['id'].isin(list(folds[0][0]))]

In [ ]:
for i in tqdm(range(200000, 300000)):
    row = train_df.iloc[i]
    id_ = row['id']
    patch_x = row['patch_x']
    patch_y = row['patch_y']

    index = train_df.iloc[i]['Unnamed: 0']

    reduced_matrix = np.load("/raid/HEST-1K/071524_hvg_matrix/hvg_matrix_{0}.npy".format(id_)).T
    image_features = np.load('/raid/mpleasure/data_deepstorage/st_projects/visium/uni_embeds_224_20x_normalized/{0}/{1}_{2}.npy'.format(id_, patch_x, patch_y))
    if image_features.shape[0] != 1024 or reduced_matrix[index].shape[0] != 7968:
        print('yeeees', i)
#     item = {}
#     item['image_features'] = torch.tensor(image_features).float() #color channel first, then XY
#     item['reduced_expression'] = torch.tensor(reduced_matrix[index]).float()  #cell x features (3467)
    # print(idx, len(np.unique(reduced_matrix[index])))

 75%|████████████████████████████████████████████████████████████████████████████████████████████▍                              | 75135/100000 [1:29:08<30:23, 13.63it/s]

In [10]:
import numpy as np
import pandas as pd
from glob import glob
folds = np.load('/raid/eredekop/071024_ST/data/folds.npy', allow_pickle=True)
df = pd.read_csv('/raid/mpleasure/data_deepstorage/st_projects/visium/all_coords_visium_data.csv')
val_df = df[df['id'].isin(list(folds[0][1]))]
ids = folds[0][1]

In [12]:
for item in ids:
    print(item, len(glob('/raid/HEST-1K/071524_hvg_matrix/*{0}*'.format(item))))

TENX71 1
TENX70 1
TENX65 1
TENX49 1
TENX40 1
TENX39 1
TENX15 1
ZEN47 1
ZEN46 1
ZEN44 1
ZEN40 1
ZEN39 1
ZEN37 1
INT35 1
INT28 1
INT24 1
INT23 1
INT19 1
INT12 1
INT11 1
INT10 1
INT6 1
INT2 10
MEND161 1
MEND160 1
MEND158 1
MEND157 1
MEND148 1
MEND143 1
MEND142 1
MEND91 1
MEND88 1
MEND86 1
MEND85 1
MEND60 1
MEND59 1
MEND54 1
MEND49 1
MEND48 1
MEND39 1
MEND38 1
MEND37 1
MEND33 1
NCBI855 0
NCBI854 0
NCBI828 1
NCBI827 1
NCBI776 1
NCBI771 1
NCBI714 1
NCBI712 1
NCBI709 1
NCBI705 1
NCBI703 1
NCBI700 1
NCBI696 1
NCBI691 1
NCBI682 1
NCBI674 1
NCBI643 1
NCBI592 1
NCBI570 1
NCBI567 1
NCBI562 1
NCBI537 1
NCBI535 1
NCBI526 1
NCBI521 1
NCBI510 1
NCBI505 1
NCBI504 1
NCBI496 1
NCBI494 1
NCBI492 1
NCBI491 1
NCBI486 1
NCBI483 1
NCBI481 1
NCBI476 1
NCBI470 1
NCBI468 1
NCBI467 1
NCBI465 1
NCBI463 1
NCBI1 1
MISC32 1
MISC29 1
MISC24 1
MISC21 1
MISC19 1
MISC17 1
MISC16 1


In [14]:
val_df = val_df[val_df['id'] !='NCBI855']
val_df = val_df[val_df['id'] !='NCBI854']

In [13]:
val_df

,Unnamed: 0,id,bead,patch_x,patch_y,bead_center_x,bead_center_y
125732,0,TENX71,0,31860,12555,31972.0,12667.0
125733,1,TENX71,1,8887,26522,8999.0,26634.0
125734,2,TENX71,2,30561,5290,30673.0,5402.0
125735,3,TENX71,3,12317,12292,12429.0,12404.0
125736,4,TENX71,4,5542,6310,5654.0,6422.0
...,...,...,...,...,...,...,...
1283991,3149,MISC16,3149,16784,6390,16896.0,6502.0
1283992,3150,MISC16,3150,18211,7729,18323.0,7841.0
1283993,3151,MISC16,3151,14362,12887,14474.0,12999.0
1283994,3152,MISC16,3152,12716,10401,12828.0,10513.0
